In [17]:
import torch
from torch import nn

In [18]:
from syft.lib.tensor.tensorbase import SyftTensor

In [19]:
x = SyftTensor.FloatTensor([1,2,3])

In [4]:
def foo(x, y):
    return 2 * x + y

traced_foo = torch.jit.trace(foo, (torch.rand(3), torch.rand(3)))

@torch.jit.script
def bar(x):
    return traced_foo(x, x)

In [5]:
traced_foo, bar

(<torch.jit.ScriptFunction at 0x7fe36b518d70>,
 <torch.jit.ScriptFunction at 0x7fe36b518c50>)

In [6]:
def fuu(x):
    return traced_foo(x,x)

In [12]:
fuu_scripted = torch.jit.script(fuu, torch.rand(3))

In [13]:
fuu_scripted(torch.tensor([1,2]))

tensor([3, 6])

In [11]:
x = torch.tensor([1,2])
y = torch.tensor([3,4])

In [8]:
bar(x)

tensor([3, 6])

In [61]:
def set_model_params(module, params_list, start_param_idx: int=0):
    """ Set params list into model recursively
    """
    param_idx = start_param_idx

#     for name, param in module._parameters.items():
#         module._parameters[name] = params_list[param_idx]
#         param_idx += 1

#     for name, child in module._modules.items():
#         if child is not None:
#             param_idx = set_model_params(child, params_list, param_idx)

#     return param_idx

In [110]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 392)
        self.fc2 = nn.Linear(392, 10)

    def forward(self, x):
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        return x

# model = torch.jit.script(Net())
model =Net()

In [111]:
def softmax_cross_entropy_with_logits(logits, targets, batch_size):
    """ Calculates softmax entropy
        Args:
            * logits: (NxC) outputs of dense layer
            * targets: (NxC) one-hot encoded labels
            * batch_size: value of N, temporarily required because Plan cannot trace .shape
    """
    # numstable logsoftmax
    norm_logits = logits - logits.max()
    log_probs = norm_logits - norm_logits.exp().sum(dim=1, keepdim=True).log()
    # NLL, reduction = mean
    return -(targets * log_probs).sum() / batch_size

In [112]:
model._parameters

OrderedDict()

In [113]:
def naive_sgd(param, **kwargs):
    return param - kwargs['lr'] * param.grad

In [114]:
@torch.jit.script
def training_plan(X, y, batch_size, lr, model_params):
    # inject params into model
#     set_model_params(model, model_params)

    # forward pass
    logits = model.forward(X)
    
    # loss
    loss = softmax_cross_entropy_with_logits(logits, y, batch_size)

    # backprop
    loss.backward()

    # step
    updated_params = [
        naive_sgd(param, lr=lr)
        for param in model_params
    ]
    
    # accuracy
    pred = th.argmax(logits, dim=1)
    target = th.argmax(y, dim=1)
    acc = pred.eq(target).sum().float() / batch_size

    return (
        loss,
        acc,
        *updated_params
    )


RuntimeError: 
attribute lookup is not defined on python value of type 'Net':
  File "<ipython-input-114-6608da59597e>", line 7

    # forward pass
    logits = model.forward(X)
             ~~~~~~~~~~~~~ <--- HERE
    
    # loss


In [80]:
model = my_script_module = torch.jit.trace(Net(), (,))

SyntaxError: invalid syntax (<ipython-input-80-f44551d57a08>, line 1)

In [74]:
model_params = [param.data for param in model.parameters()] 

x = torch.rand((32, 784,392))
y = torch.randint(0, 10, (32,10))
bs, lr = 32, 0.01

In [75]:
torch.jit.script(training_plan, (x, y, bs, lr, model_params))

RuntimeError: 
Cannot call a ScriptModule that is not a submodule of the caller:
  File "<ipython-input-48-76a76cc55314>", line 3
def training_plan(X, y, batch_size, lr, model_params):
    # inject params into model
    set_model_params(model, model_params)
                     ~~~~~ <--- HERE

    # forward pass
